## Data 690 (AI) Final Project
## Facial Recognition Demo
## Yunpeng Li

In [60]:
import os
from os import listdir
from os.path import isfile, join
import cv2
import numpy as np
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from PIL import Image
from tensorflow.keras.preprocessing.image import load_img, save_img, img_to_array
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

## Function to setup the directories we'll be storing our images:

In [61]:
def makedir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
        return None
    else:
        pass


## Function for extracting person faces from original photo in a source directory and save them to a destination directory:

In [85]:
def extractFacesFromPictures(source_dir, dest_dir):
    
    makedir(dest_dir)
    
    # Loading HAARCascade Face Detector 
    faceDetector = cv2.CascadeClassifier('Haarcascades/haarcascade_frontalface_default.xml')
    
    imageNames = [f for f in listdir(source_dir) if isfile(join(source_dir, f))]
    
    for imageName in imageNames:
        print(imageName)
        image = cv2.imread(source_dir+imageName)
        
        cv2.imshow('imageName', image) 
        cv2.waitKey(0)
        
        face_info = faceDetector.detectMultiScale(image, 1.3, 5)
        
        print(face_info)
        for (x,y,w,h) in face_info:
            face = image[y:y+h, x:x+w]
            roi = cv2.resize(face, (128, 128), interpolation = cv2.INTER_CUBIC)
        
            cv2.imshow("face", roi)
            
            path = dest_dir + "face_" + imageName 
            cv2.imwrite(path, roi) #Save the extracted faces into the destination directory
            
            cv2.waitKey(0)
        
    cv2.destroyAllWindows()

In [124]:
extractFacesFromPictures('./images/', './faces/')

Britney Spears.jpg
[[403 307 830 830]]
Dwayne Johnson (Rock).jpg
[[ 67  63 185 185]]
Hillary.jpg
[[392 612 836 836]]
Jenny.jpg
[[ 57  86 192 192]]
Kim Jong Un.jpg
[[ 80  74 189 189]]
Lady Gaga.jpg
[[155  53 413 413]]
Madonna.jpg
[[507 171 516 516]]
Trump.jpg
[[1448  457  890  890]]
Yunpeng.jpg
[[664 252 218 218]]


## Use VGGFaceModel

In [64]:
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(Convolution2D(4096, (7, 7), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(4096, (1, 1), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(2622, (1, 1)))
model.add(Flatten())
model.add(Activation('softmax'))

#TODO
#Download pretrained weights from https://drive.google.com/file/d/1CPSeum3HpopfomUEK1gybeuIVoeJT_Eo/view?usp=sharing

#from tensorflow.keras.models import model_from_json
model.load_weights('vgg_face_weights.h5')

#Remove last Softmax layer and get model upto last flatten layer
vgg_face_descriptor = Model(inputs=model.layers[0].input, outputs=model.layers[-2].output)

vggFaceModel = vgg_face_descriptor

## VggFaceModel architecture

In [65]:
vggFaceModel.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_26_input (Inp [(None, 224, 224, 3)]     0         
_________________________________________________________________
zero_padding2d_26 (ZeroPaddi (None, 226, 226, 3)       0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 224, 224, 64)      1792      
_________________________________________________________________
zero_padding2d_27 (ZeroPaddi (None, 226, 226, 64)      0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 112, 112, 64)      0         
_________________________________________________________________
zero_padding2d_28 (ZeroPaddi (None, 114, 114, 64)      0   

## Function for loading image from path and resizes it

In [66]:
def preprocess_image(image_path):
    """Loads image from path and resizes it"""
    img = load_img(image_path, target_size=(224, 224))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img

## Compare images by Cosine Similiarity:

In [67]:
def calculateCosineSimilarity(source_representation, test_representation):
    a = np.matmul(np.transpose(source_representation), test_representation)
    b = np.sum(np.multiply(source_representation, source_representation))
    c = np.sum(np.multiply(test_representation, test_representation))
    return 1 - (a / (np.sqrt(b) * np.sqrt(c)))

## Create presentation of each extraced face by VGGFaceModel prediction:

In [125]:
all_faces = dict()

for file in listdir('./faces/'):
    
    print(file)
    person_face, extension = file.split(".")
    all_faces[person_face] = vggFaceModel.predict(preprocess_image('./faces/%s.jpg' % (person_face)))[0,:]


face_Britney Spears.jpg
face_Dwayne Johnson (Rock).jpg
face_Hillary.jpg
face_Jenny.jpg
face_Kim Jong Un.jpg
face_Lady Gaga.jpg
face_Madonna.jpg
face_Trump.jpg
face_Yunpeng.jpg


In [126]:
all_faces

{'face_Britney Spears': array([ 1.60496   , -1.5575426 , -0.14216928, ...,  3.4180443 ,
        -0.42121536,  0.14598373], dtype=float32),
 'face_Dwayne Johnson (Rock)': array([ 3.9746056 , -2.6902373 , -0.30208504, ..., -1.9033624 ,
         0.47788623,  0.14036633], dtype=float32),
 'face_Hillary': array([ 1.0320519, -2.672826 , -3.1798182, ..., -2.312236 ,  0.9663745,
         1.2726313], dtype=float32),
 'face_Jenny': array([ 0.88640165,  1.1435761 ,  0.45120814, ..., -2.4543052 ,
         2.3160427 ,  3.7199552 ], dtype=float32),
 'face_Kim Jong Un': array([ 0.81118304, -1.084969  ,  0.1463707 , ..., -2.7813516 ,
         4.306972  ,  0.53672796], dtype=float32),
 'face_Lady Gaga': array([ 1.7031907 , -4.33134   , -2.6338637 , ..., -1.8551259 ,
         0.06040401,  3.3642864 ], dtype=float32),
 'face_Madonna': array([ 0.68289185, -0.9796021 ,  1.4411439 , ..., -4.76454   ,
         2.0778015 ,  2.0321078 ], dtype=float32),
 'face_Trump': array([ 2.4377742 , -0.18090715, -1.643119

## Function for using cv2.VideoCapture to recognize face similar to the face images stored in the face_database:

In [70]:
def recognize_face(face_database, video_capture):
    
    # Loading HAARCascade Face Detector 
    faceDetector = cv2.CascadeClassifier('Haarcascades/haarcascade_frontalface_default.xml')

    while(True):
        ret, img = video_capture.read()
        faces = faceDetector.detectMultiScale(img, 1.3, 5)

        for (x,y,w,h) in faces:

            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2) #draw rectangle to main image

            detected_face = img[int(y):int(y+h), int(x):int(x+w)] #crop detected face
            detected_face = cv2.resize(detected_face, (224, 224)) #resize to 224x224

            img_pixels = image.img_to_array(detected_face)
            img_pixels = np.expand_dims(img_pixels, axis = 0)
            img_pixels /= 255

            captured_representation = vggFaceModel.predict(img_pixels)[0,:]

            print(captured_representation)

            found = False

            # Compare captured face with the stored face
            for face in face_database:
                representation = face_database[face]
                person_name=face[5:]

                similarity = calculateCosineSimilarity(representation, captured_representation)

                if(similarity < 0.25):
                    cv2.putText(img, person_name, (int(x+w+15), int(y-12)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                    found = True
                    break

            #connect face and text

            cv2.line(img,(int((x+x+w)/2),y+15),(x+w,y-20),(255, 0, 0),1)
            cv2.line(img,(x+w,y-20),(x+w+10,y-20),(255, 0, 0),1)

            if(found == False): #if found image is not in our people database
                cv2.putText(img, 'unknown', (int(x+w+15), int(y-12)), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)


        cv2.imshow('img',img)

        if cv2.waitKey(1) == 13: #13 is the Enter Key
            break

    video_capture.release()
    cv2.destroyAllWindows()

### Use webcam to detect and capture face in real time and compare it with the stored faces so as to recogize face:  

In [127]:
#Open Webcam
camera = cv2.VideoCapture(0)

detect_face(all_faces, camera)

[ 0.01040188  0.00106935  0.00314128 ... -0.01295905  0.01894774
  0.01196851]
[ 0.01804194  0.00712961  0.00818555 ... -0.01505877  0.02321138
  0.00406144]
[ 0.01769135  0.005941    0.00462554 ... -0.01580085  0.02450629
  0.00588839]
[ 0.01719665  0.00710686  0.0076104  ... -0.01558237  0.02208028
  0.00426522]
[ 0.01786124  0.00679336  0.00428505 ... -0.01485613  0.02257449
  0.00428969]
[ 0.01829234  0.00800912  0.00765472 ... -0.0154951   0.02475243
  0.00547959]
[ 0.01561677  0.01027791 -0.00202604 ... -0.01717918  0.02423229
  0.0119628 ]
[ 0.02279359  0.013145    0.00650536 ... -0.01672066  0.02098855
  0.02228116]
[ 0.02058489  0.02043145  0.00314734 ... -0.02123448  0.0291664
  0.01944316]
[ 0.02091859  0.01752253 -0.00104183 ... -0.0208318   0.03015089
  0.01601984]
[ 0.01523335  0.01369027  0.0007772  ... -0.01969828  0.02599016
  0.01723714]
[ 0.01940145  0.0157293   0.00185146 ... -0.02004239  0.02689217
  0.01685565]
[ 0.01871081  0.01306288  0.00258835 ... -0.02064083 

### Analyze recorded video to recognize faces in the video:  

In [139]:
video_capture_1 = cv2.VideoCapture('./videos/1.mp4')

detect_face(all_faces, video_capture_1)

[ 0.00296052 -0.00072332  0.00210925 ... -0.00253696  0.00412182
  0.02561892]
[ 0.00934224 -0.00051054  0.00173485 ... -0.02042681  0.0204794
  0.0099755 ]
[ 0.00217472 -0.00164675  0.00177252 ... -0.00423024  0.00332869
  0.02605653]
[ 0.01433773  0.00182296  0.00555621 ... -0.02348037  0.02430162
  0.01101995]
[ 0.00286887 -0.00174106  0.0008247  ... -0.00455105  0.00413421
  0.0259352 ]
[ 0.01175528  0.00115652  0.00395784 ... -0.02295435  0.02260291
  0.01046151]
[ 0.00367001 -0.00241869 -0.0006467  ... -0.00362563  0.00508426
  0.02567327]
[ 0.01161902  0.00201731  0.00372995 ... -0.02373979  0.02244275
  0.01006952]
[ 0.0028968  -0.0016458   0.00092645 ... -0.00330563  0.00419714
  0.02647534]
[ 0.0084905  -0.00100416  0.00177778 ... -0.02313355  0.02160888
  0.01026437]
[ 0.00324872 -0.00309848  0.00022815 ... -0.00373952  0.00392483
  0.02759625]
[ 0.01021929  0.00051542  0.00245499 ... -0.02339717  0.02276633
  0.01082117]
[ 0.00421457 -0.00110154  0.00165922 ... -0.00352115 

In [120]:
video_capture_2 = cv2.VideoCapture('./videos/2.mp4')

detect_face(all_faces, video_capture_2)

[-0.00204057  0.00295777 -0.00256422 ... -0.00439538  0.00399822
  0.00665387]
[ 0.00066078  0.00366082 -0.00294143 ... -0.00251856  0.00107463
  0.00347667]
[ 0.00376614  0.00543297 -0.00363034 ...  0.00814521 -0.00106358
  0.00525908]
[ 0.00315803  0.00149208 -0.0046159  ...  0.00785645 -0.00203934
  0.00371513]
[ 0.00326062  0.00048386 -0.00281422 ...  0.01137358 -0.00629338
 -0.00102162]
[ 0.00358567  0.00093809 -0.00363676 ...  0.00988024 -0.00555493
  0.00036617]
[ 0.00886141 -0.0020516  -0.00953427 ...  0.00418702  0.01173882
  0.0176084 ]
[ 0.00735998  0.00237411 -0.00332455 ...  0.01250444 -0.00498478
  0.00503373]
[ 0.01218766 -0.01426951 -0.01216068 ... -0.00154649  0.00614659
  0.01182063]
[ 0.00464262  0.00117292 -0.00215564 ...  0.01094957 -0.00627829
  0.00054007]
[ 0.01244994 -0.01671246 -0.01436989 ... -0.00226698  0.00693272
  0.00834761]
[ 0.00436163  0.00114548 -0.00357897 ...  0.01047354 -0.00587068
  0.00032705]
[ 0.01334499 -0.01789414 -0.01456963 ... -0.00253701

In [136]:
video_capture_3 = cv2.VideoCapture('./videos/3.mp4')

detect_face(all_faces, video_capture_3)

[ 0.00284104 -0.00774925  0.00690357 ... -0.01010167  0.01483792
  0.00810243]
[ 0.00397602  0.00715183 -0.00297459 ... -0.00515174  0.00111554
  0.00359487]
[ 0.00301588 -0.0078433   0.00559551 ... -0.01057601  0.01412831
  0.00805489]
[ 0.00489471  0.00601196 -0.00580766 ... -0.00187974 -0.00241028
  0.00062992]
[ 0.00799976  0.00476745 -0.00421184 ...  0.00028382 -0.00072314
  0.00098995]
[ 0.0038297  -0.00919363  0.00696561 ... -0.00919779  0.01425986
  0.00731399]
[ 0.00725941  0.00503019 -0.0044192  ... -0.00362337  0.00010075
  0.00281231]
[ 0.00201305 -0.01030707  0.00652388 ... -0.01187971  0.01503764
  0.00770207]
[ 0.00750658  0.00682887 -0.00273739 ... -0.00193884 -0.00101493
  0.00188053]
[ 0.00321054 -0.00972073  0.00922822 ... -0.01198062  0.01535834
  0.00775328]
[ 0.00151104 -0.01070017  0.00759787 ... -0.01275997  0.01478824
  0.00770627]
[ 0.00709849  0.0056701  -0.00211996 ... -0.00139352 -0.00164533
  0.00154553]
[ 0.00160525 -0.01158241  0.00710596 ... -0.01302548

## Release camera or video caputure:

In [140]:
camera.release()
video_capture_1.release()
video_capture_2.release()
video_capture_3.release()

In [141]:
cv2.destroyAllWindows()